# Routing Topology: OSPF using FRRouting

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Import the FABlib Library


In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/work/fabric_config/tokens.json
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Bastion Username,pruth_0031379841
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create the Experiment Slice

The following creates private layer 2 networks on three sites including OSPF gateway routers that propagate routes across the topology. 


In [2]:
slice_name = 'OSPF_Routing_Topology2'

sites= fablib.get_random_sites(count=4)
print(f"Sites: {sites}")

router_base_name='router'
router_link_base_name='router_link'

node_base_name='node'
local_network_base_name='net_local'

site_node_count=2



Sites: ['NCSA', 'UTAH', 'DALL', 'CLEM']


In [3]:

slice = fablib.new_slice(name=slice_name)

# Create Routers
routers = []
for i, site in enumerate(sites):
    router_info = {}
    router_info['name'] = f"{router_base_name}{i+1}"
    local_subnet = IPv4Network(f"192.168.{i+1}.0/24")
    local_gateway = IPv4Address(f"192.168.{i+1}.1")

    router_info['node'] = router = slice.add_node(name=router_info['name'], site=site)
    router_info['link_iface_local'] = router.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]
    router_info['link_iface_local'].set_mode('fablib')

    
    router_info['local_net'] = slice.add_l2network(name=f'{local_network_base_name}{i+1}', subnet=local_subnet, gateway=local_gateway)
    router_info['local_net'].add_interface(router_info['link_iface_local'])
    router_info['link_iface_local'].set_ip_addr(local_gateway)

    router.enable_docker()
    
    routers.append(router_info)

# Create Links between routers (ring)
links = []
for i, site in enumerate(sites):
    link_info = {}
    link_info['name'] = f'{router_link_base_name}{i+1}'
    link_subnet = IPv4Network(f"192.168.10{i+1}.0/24")

    router1 = routers[i]
    router2 = routers[(i+1)%len(sites)]

    iface1 = router1['node'].add_component(model='NIC_Basic', name=link_info['name']).get_interfaces()[0]
    iface2 = router2['node'].add_component(model='NIC_Basic', name=link_info['name']).get_interfaces()[0]
    
    iface1.set_mode('auto')
    iface2.set_mode('auto')
    
    link = slice.add_l2network(name=link_info['name'], 
                               subnet=IPv4Network(f"192.168.10{i+1}.0/24"),
                               interfaces=[iface1,iface2] 
                              )
    links.append(link)
    
slice_id = slice.submit()



Retry: 8, Time: 668 sec


ID,468b7680-6c67-467c-b745-a25202aa9034
Name,OSPF_Routing_Topology2
Lease Expiration (UTC),2023-02-20 21:11:35 +0000
Lease Start (UTC),2023-02-19 21:11:36 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
b1ffa84d-e816-4a5c-97a6-19b1da4e9364,router1,2,8,10,default_rocky_8,qcow2,ncsa-w3.fabric-testbed.net,NCSA,rocky,2620:0:c80:1001:f816:3eff:fe1c:337c,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:0:c80:1001:f816:3eff:fe1c:337c docker exec -it fabric_multitool bash,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
91bf6436-7914-49d9-b6a1-530144e4fbbf,router2,2,8,10,default_rocky_8,qcow2,utah-w4.fabric-testbed.net,UTAH,rocky,2001:1948:417:7:f816:3eff:febd:f6f3,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:1948:417:7:f816:3eff:febd:f6f3 docker exec -it fabric_multitool bash,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1d91071f-4a5a-4c93-ac54-e59715377317,router3,2,8,10,default_rocky_8,qcow2,dall-w1.fabric-testbed.net,DALL,rocky,2001:400:a100:3000:f816:3eff:fea8:212f,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3000:f816:3eff:fea8:212f docker exec -it fabric_multitool bash,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
bc001a44-7197-42c1-a9e7-24be27f292fc,router4,2,8,10,default_rocky_8,qcow2,clem-w2.fabric-testbed.net,CLEM,rocky,2620:103:a006:12:f816:3eff:fee1:a8b,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:103:a006:12:f816:3eff:fee1:a8b docker exec -it fabric_multitool bash,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
2824f7b0-2a4c-48db-952c-cea0d670dc35,net_local1,L2,L2Bridge,NCSA,192.168.1.0/24,192.168.1.1,Active,
c73a56b7-caaa-489d-8697-ceaa30209363,net_local2,L2,L2Bridge,UTAH,192.168.2.0/24,192.168.2.1,Active,
21ba4029-8416-423a-8d07-4fd0022abbb6,net_local3,L2,L2Bridge,DALL,192.168.3.0/24,192.168.3.1,Active,
07e701ff-5157-4994-9cba-7dd852c6a80b,net_local4,L2,L2Bridge,CLEM,192.168.4.0/24,192.168.4.1,Active,
af405b76-2213-42fe-a886-83cca4c10641,router_link1,L2,L2STS,None,192.168.101.0/24,None,Active,
f393a5d3-52a2-48be-988c-7070a8a93b09,router_link2,L2,L2STS,None,192.168.102.0/24,None,Active,
45665eef-929a-47a1-ad67-a49aa24dc32e,router_link3,L2,L2STS,None,192.168.103.0/24,None,Active,
b8479594-29b7-4539-ab8a-2aec3856ab89,router_link4,L2,L2STS,None,192.168.104.0/24,None,Active,


Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address
router1-router_link1-p1,router1,router_link1,100,auto,,06:85:CD:D6:13:B3,eth1,eth1,192.168.101.1
router1-router_link4-p1,router1,router_link4,100,auto,,0A:12:62:FD:B9:81,eth2,eth2,192.168.104.1
router1-nic_local-p1,router1,net_local1,100,fablib,,0A:A6:98:56:67:38,eth3,eth3,192.168.1.1
router2-router_link1-p1,router2,router_link1,100,auto,,02:4C:BD:CA:16:E8,eth1,eth1,192.168.101.2
router2-nic_local-p1,router2,net_local2,100,fablib,,0E:F0:9C:F6:F3:1D,eth3,eth3,192.168.2.1
router2-router_link2-p1,router2,router_link2,100,auto,,0E:1F:C4:CA:81:4E,eth2,eth2,192.168.102.2
router3-router_link2-p1,router3,router_link2,100,auto,,06:37:96:B6:FB:74,eth1,eth1,192.168.102.1
router3-nic_local-p1,router3,net_local3,100,fablib,,12:31:A0:9C:36:BC,eth2,eth2,192.168.3.1
router3-router_link3-p1,router3,router_link3,100,auto,,12:68:1B:76:CE:21,eth3,eth3,192.168.103.2
router4-router_link3-p1,router4,router_link3,100,auto,,0A:1D:CC:D9:6D:0B,eth3,eth3,192.168.103.1



Time to print interfaces 681 seconds


## Observe the Slice's Attributes


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

Pick the subnet for the local networks.  The /24 subnets can support up to 254 locally connected nodes plus the gateway. 


In [ ]:
try:
    # Local Subnets
    net_local1_subnet = IPv4Network("192.168.1.0/24")
    net_local1_available_ips = list(net_local1_subnet)[1:]
    net_local1_gateway = net_local1_available_ips.pop(0)
        
    net_local2_subnet = IPv4Network("192.168.2.0/24")
    net_local2_available_ips = list(net_local2_subnet)[1:]
    net_local2_gateway = net_local2_available_ips.pop(0)


    net_local3_subnet = IPv4Network("192.168.3.0/24")
    net_local3_available_ips = list(net_local3_subnet)[1:]
    net_local3_gateway = net_local3_available_ips.pop(0)

    print(f"Site1: subnet: {net_local1_subnet}, gateway: {net_local1_gateway}")
    print(f"Site2: subnet: {net_local2_subnet}, gateway: {net_local2_gateway}")
    print(f"Site3: subnet: {net_local3_subnet}, gateway: {net_local3_gateway}")

except Exception as e:
    print(f"Exception: {e}")

### Configure Router IPs

Add the IPs to the corrisponing interfaces.

In [ ]:
try:
    # Config Router1 IPs
    router1 = slice.get_node(name=router1_name)  
    router1_iface1 = router1.get_interface(network_name=route_link1_name)  
    router1_iface2 = router1.get_interface(network_name=route_link3_name)  
    router1_local_iface = router1.get_interface(network_name=site1_local_net_name)     
    router1_iface1.ip_addr_add(addr=route_link1_addr1, subnet=route_link1_subnet)
    router1_iface2.ip_addr_add(addr=route_link3_addr2, subnet=route_link3_subnet)
    router1_local_iface.ip_addr_add(addr=net_local1_gateway, subnet=net_local1_subnet)
    
    # Config Router2 IPs
    router2 = slice.get_node(name=router2_name)  
    router2_iface1 = router2.get_interface(network_name=route_link2_name)  
    router2_iface2 = router2.get_interface(network_name=route_link1_name)  
    router2_local_iface = router2.get_interface(network_name=site2_local_net_name)     
    router2_iface1.ip_addr_add(addr=route_link2_addr1, subnet=route_link2_subnet)
    router2_iface2.ip_addr_add(addr=route_link1_addr2, subnet=route_link1_subnet)
    router2_local_iface.ip_addr_add(addr=net_local2_gateway, subnet=net_local2_subnet)
    
    # Config Router3 IPs
    router3 = slice.get_node(name=router3_name) 
    router3_iface1 = router3.get_interface(network_name=route_link3_name)  
    router3_iface2 = router3.get_interface(network_name=route_link2_name)  
    router3_local_iface = router3.get_interface(network_name=site3_local_net_name)     
    router3_iface1.ip_addr_add(addr=route_link3_addr1, subnet=route_link3_subnet)
    router3_iface2.ip_addr_add(addr=route_link2_addr2, subnet=route_link2_subnet)
    router3_local_iface.ip_addr_add(addr=net_local3_gateway, subnet=net_local3_subnet)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()
    

(Optional) Print the network interface configuration for each router.

In [ ]:
try:
    for router in [router1, router2, router3]:
        print(f'{router.get_name()}:')
        stdout, stderr = router.execute(f'ip addr list')
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

### Configure Local Node IPs

Configure the local nodes with addresses from the local subnet available address list. Add routes to the other local subnets via the local gateway.  

Add the router link subnets if you want to access the routers (i.e. if you want to `ping` the routers or find paths with `tracepath`)

Collect a list of local dataplane IPs to target for testing

In [ ]:
local_dataplane_ips = {}

In [ ]:
try:    
    #Create Site1 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_site1_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local1_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site1_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local1_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local2_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=net_local3_subnet, gateway=net_local1_gateway)
        
        #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local1_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr
        
        print(f"Node {name} dataplane IP: {node_addr}")    
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
except Exception as e:
    print(f"Exception: {e}")
    

In [ ]:
try:
    #Create Site2 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_site2_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local2_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site2_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local2_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local1_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=net_local3_subnet, gateway=net_local2_gateway)
        
         #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local2_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr

        print(f"Node {name} dataplane IP: {node_addr}")
        stdout, stderr = node.execute(f'ip addr list')
        
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    #Create Site3 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_site3_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local3_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site3_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local3_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local1_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=net_local2_subnet, gateway=net_local3_gateway)
        
        #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local3_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr
        
        print(f"Node {name} dataplane IP: {node_addr}")
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')    
except Exception as e:
    print(f"Exception: {e}")

##  Configure FRRouting on each router. 

This complex configuration is handled through a bash script `frr_config.sh` that resides in the folder containing this notebook. The script is executed by, first, uploading the script with the `node.upload_file()` FABLib method.  Then the script is executed using the `node.execute()` FABLib method.  Note that the script passes the OS interfaces names and configured IPs as arguments from the notebook to the script.  

These scripts take a while to run. You may wish to use a separate terminal window to ssh to the routers and tail the log file to watch the progress with: `tail -F frr_config.log`.

In [12]:
import os
for router in routers:
    print(f"{router['name']}")

    fablib_router = slice.get_node(router['name'])
    
    fablib_router.upload_directory(f'{os.environ["HOME"]}/work/docker_containers','.')
    fablib_router.execute(f"cd docker_containers/fabric_frrouting ; docker compose up -d", quiet=True, output_file=f"{fablib_router.get_name()}.log");
        

router1
router2
router3
router4


In [11]:
try: 
    #thread1 = execute.thread()
    #stdout, stderr = thread1.result()
    
    # Config Router1
    print('Config Router1')
    router1.upload_file('./frr_config.sh','frr_config.sh')
    router1_config_thread = router1.execute_thread(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router1_iface1.get_os_interface()} {route_link1_addr1} {router1_iface2.get_os_interface()} {route_link3_addr2} {router1_local_iface.get_os_interface()} {net_local1_gateway} 192.168.0.0')

    # Config Router2
    print('Config Router2')
    router2.upload_file('./frr_config.sh','frr_config.sh')   
    router2_config_thread = router2.execute_thread(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router2_iface1.get_os_interface()} {route_link2_addr1} {router2_iface2.get_os_interface()} {route_link1_addr2} {router2_local_iface.get_os_interface()} {net_local2_gateway} 192.168.0.0')
    
    # Config Router3
    print('Config Router3')
    router3.upload_file('./frr_config.sh','frr_config.sh')
    router3_config_thread = router3.execute_thread(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router3_iface1.get_os_interface()} {route_link3_addr1} {router3_iface2.get_os_interface()} {route_link2_addr2} {router3_local_iface.get_os_interface()} {net_local3_gateway} 192.168.0.0')

    #Join Threads
    print(f"Joining Threads")
    stdout, stderr = router1_config_thread.result() 
    print(f"Router1: ", stdout, stderr)
    stdout, stderr = router2_config_thread.result() 
    print(f"Router2: ", stdout, stderr)
    stdout, stderr = router3_config_thread.result() 
    print(f"Router3: ", stdout, stderr)

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()            
     

Config Router1
Exception: 'dict' object has no attribute 'upload_file'


NameError: name 'traceback' is not defined

## Run the Experiment

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [ ]:
try:
    source_node_name =  f'{node_base_name}_site1_1'
   
    source_node = slice.get_node(name=source_node_name)
    for node_name,target_ip in local_dataplane_ips.items():
        print(f"Testing target node: {node_name}, target IP: {target_ip}")
    
        stdout, stderr = node.execute(f'ping -c 5 {target_ip}')

        stdout, stderr = node.execute(f'tracepath {target_ip}')
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice.save("ospf.graphml")
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")